In [5]:
from pyspark.sql import SparkSession

In [ ]:
SparkSession.builder.master

In [8]:
spark = SparkSession.builder.master("local").appName("Word Count").config("spark.some.config.option", "1").getOrCreate()

WindowsError: [Error 2] 

In [4]:
from __future__ import division
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
from sklearn.ensemble import IsolationForest

In [2]:
train=pd.read_csv('train.csv',na_values=-1)
test=pd.read_csv('test.csv',na_values=-1)

In [3]:
train.head()

,id,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
0,7,0,2,2,5,1,0,0,1,0,...,9,1,5,8,0,1,1,0,0,1
1,9,0,1,1,7,0,0,0,0,1,...,3,1,1,9,0,1,1,0,1,0
2,13,0,5,4,9,1,0,0,0,1,...,4,2,7,7,0,1,1,0,1,0
3,16,0,0,1,2,0,0,1,0,0,...,2,2,4,9,0,0,0,0,0,0
4,17,0,0,2,0,1,0,1,0,0,...,3,1,1,3,0,0,0,1,1,0


In [9]:
train_copy = train.replace(-1, np.nan)

In [10]:
for col in train_copy.columns:
    port = sum(train_copy[col].isnull()) / len(train_copy)
    if port >= 0.6:
        print col, port

ps_car_03_cat 0.690898368984


In [ ]:
dummy_col = train.columns['bin' in item for item in train.columns]
bin_dummy = pd.get_dummies(train[dummy_col])


In [3]:
num_col = train.columns[['cat' not in item and 'bin' not in item for item in train.columns]]
iso_train = train[num_col]
iso_train.drop(['id', 'target'], axis=1, inplace=True)
isf = IsolationForest(max_samples = 10*10**4, random_state=42, contamination=0.1)
isf.fit(iso_train)
y_pred_train = isf.predict(iso_train)
iso_train['is_outlier'] = y_pred_train

C:\Users\KrystalU\Anaconda2\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\KrystalU\Anaconda2\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [4]:
train = train[iso_train['is_outlier'] == 1]
features = train.drop(['id','target'], axis=1).values
targets = train.target.values

In [5]:
unwanted = train.columns[train.columns.str.startswith('ps_calc_')]

In [6]:
train = train.drop(unwanted, axis=1)  
test = test.drop(unwanted, axis=1) 

In [7]:
kfold = 5
skf = StratifiedKFold(n_splits=kfold, random_state=42)

In [8]:
X = train.drop(['id', 'target'], axis=1).values
y = train.target.values
test_id = test.id.values
test = test.drop('id', axis=1)

In [9]:
sub = pd.DataFrame()
sub['id'] = test_id
sub['target'] = np.zeros_like(test_id)

In [10]:
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from time import time

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify = y)

In [13]:
param_dist = {
    'model__min_child_weight': [5,7,9],
    'model__max_depth': [5, 6, 7],
    'model__max_delta_step': [1, 1.5, 1.8, 2.0], 
    'model__colsample_bytree': [0.6, 0.8],
    'model__subsample': [0.6, 0.8],
    'model__learning_rate': [0.05, 0.1, 0.15],
    'model__gamma': [0.5, 0.65, 0.8],
    'model__n_estimators' : [500, 750, 1000, 1500],
    }

# fit_dict = {"eval_set":[(X_test, y_test)],
#             "early_stopping_rounds": 70,
#             "eval_metric": gini_xgb}

In [16]:
# run randomized search
from sklearn.metrics import make_scorer
from sklearn import pipeline

def gini(solution, submission):
    df = zip(solution, submission, range(len(solution)))
    df = sorted(df, key=lambda x: (x[1],-x[2]), reverse=True)
    rand = [float(i+1)/float(len(df)) for i in range(len(df))]
    totalPos = float(sum([x[0] for x in df]))
    cumPosFound = [df[0][0]]
    for i in range(1,len(df)):
        cumPosFound.append(cumPosFound[len(cumPosFound)-1] + df[i][0])
    Lorentz = [float(x)/totalPos for x in cumPosFound]
    Gini = [Lorentz[i]-rand[i] for i in range(len(df))]
    return sum(Gini)

def normalized_gini(solution, submission):
    normalized_gini = gini(solution, submission)/gini(solution, solution)
    return normalized_gini

est = pipeline.Pipeline([('model', xgb.XGBClassifier(random_state=42,objective='binary:logistic'))])

# Normalized Gini Scorer
gini_scorer = make_scorer(normalized_gini, greater_is_better = True)

# Initialize Grid Search Model
model = RandomizedSearchCV(estimator  = est,
                           param_distributions = param_dist,
                           scoring    = gini_scorer,
                           n_iter = 2,
                           verbose    = 5,
                           n_jobs     = 4,
                           cv         = 3,
                           random_state = 42)
# Fit Grid Search Model
start = time()
model.fit(train, labels)
print time()-start
# xgb_tune = xgb.XGBClassifier(random_state=42,silent=False,objective='binary:logistic')
# n_iter_search = 1
# gini_scorer = make_scorer(gini_normalized, greater_is_better=True)
# random_search = RandomizedSearchCV(estimator = xgb_tune, 
#                                    param_distributions = param_dist,
#                                    scoring = gini_scorer,
#                                    n_iter=n_iter_search, 
#                                    random_state=42, 
#                                    cv=3, 
#                                    verbose=5, 
#                                    n_jobs=8)
# start = time()
# # random_search.fit(X_train, y_train)
# print time()-start

NameError: name 'labels' is not defined

In [15]:
RandomizedSearchCV?

In [13]:
random_search.best_params_

{'colsample_bytree': 0.6,
 'gamma': 0.5,
 'learning_rate': 0.05,
 'max_delta_step': 1,
 'max_depth': 7,
 'min_child_weight': 9,
 'n_estimators': 500,
 'objective': 'binary:logistic',
 'subsample': 0.8}

In [15]:
test.head()

,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,ps_ind_10_bin,...,ps_car_07_cat,ps_car_08_cat,ps_car_09_cat,ps_car_10_cat,ps_car_11_cat,ps_car_11,ps_car_12,ps_car_13,ps_car_14,ps_car_15
0,0,1,8,1,0,0,1,0,0,0,...,1,1,2,1,65,1,0.316228,0.669556,0.352136,3.464102
1,4,2,5,1,0,0,0,0,1,0,...,1,1,0,1,103,1,0.316228,0.606320,0.358329,2.828427
2,5,1,3,0,0,0,0,0,1,0,...,1,1,2,1,29,3,0.400000,0.896239,0.398497,3.316625
3,0,1,6,0,0,1,0,0,0,0,...,1,1,2,1,40,2,0.374166,0.652110,0.381445,2.449490
4,5,1,7,0,0,0,0,0,1,0,...,1,1,2,1,101,3,0.374166,0.812914,0.385097,3.316625


In [16]:
pred_prob = random_search.best_estimator_.predict_proba(test.values)

In [18]:
pred_prob[pred_prob[:,1]>=0.5]

array([[ 0.47182536,  0.52817464],
       [ 0.42956066,  0.57043934],
       [ 0.28647822,  0.71352178],
       [ 0.40380466,  0.59619534],
       [ 0.35401535,  0.64598465],
       [ 0.49142951,  0.50857049],
       [ 0.31636304,  0.68363696],
       [ 0.35118192,  0.64881808],
       [ 0.48315603,  0.51684397],
       [ 0.42022216,  0.57977784],
       [ 0.49270606,  0.50729394],
       [ 0.24383259,  0.75616741],
       [ 0.49166304,  0.50833696],
       [ 0.4841367 ,  0.5158633 ],
       [ 0.28107172,  0.71892828],
       [ 0.47636878,  0.52363122],
       [ 0.38884741,  0.61115259],
       [ 0.42270523,  0.57729477],
       [ 0.30145675,  0.69854325],
       [ 0.4750964 ,  0.5249036 ],
       [ 0.46523005,  0.53476995],
       [ 0.4958545 ,  0.5041455 ],
       [ 0.33532912,  0.66467088]], dtype=float32)

In [20]:
sub['target'] = pred_prob[:,1]
sub.to_csv('sub3.csv',index=False)

In [21]:
random_search.cv_results_

{'mean_fit_time': array([ 2483.72218053,   806.28064259,  1121.10867508,   939.05136847,
         1500.09509913,   746.17723672,   843.28344425,  1021.78309782,
         2470.98910872,  3747.24846164,  1958.63513565,   511.23391811,
         1215.26365916,  1039.88301611,  2209.30005614,   894.51444276,
          786.30104438,  1386.93018762,  1184.51385466,  1250.93763049,
         1516.53530598,  2077.71457783,  1029.87452555,  1692.65241742,
         1024.42150887,   517.6173656 ,  2401.37014508,   974.4582595 ,
         1796.11595988,  1640.97956745,   717.34589362,   478.11443035,
         1739.01065731,  1073.26778094,  1893.16464059,   586.92967765,
         1783.71342699,  1254.60533412,  1745.02531513,  1286.18680986,
         2072.5745306 ,   745.19132082,   742.37758557,  1893.69886947,
         2703.94290106,   658.22940302,  1845.89382124,  1459.71954568,
         1994.37432416,  2002.19667665,   796.5109814 ,  2575.25857536,
          885.34942047,  1045.75941904,  1164.5